In [11]:
!pip install setuptools==58.2.0 packaging==23.2
!pip install python-terrier==0.12.1
!pip install pandas
!pip install torch==2.6.0+cu118 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git
!pip install --upgrade git+https://github.com/cmacdonald/pyterrier_bert.git
!pip install faiss-cpu --no-cache-dir

  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 1.30.1 requires packaging<22, but you have packaging 23.2 which is incompatible.


Looking in indexes: https://download.pytorch.org/whl/cu118
^C
  Cloning https://github.com/terrierteam/pyterrier_colbert.git to c:\users\konstantin-asen\appdata\local\temp\pip-req-build-g5umfp1w
  Resolved https://github.com/terrierteam/pyterrier_colbert.git to commit ba5c86c0bc8da450dee361140541f35b5349a492
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/cmacdonald/ColBERT.git (to revision v0.2) to c:\users\konstantin-asen\appdata\local\temp\pip-install-7jbxgzfq\colbert_3d6df633af30467681aa8ba52caabd0e
  Resolved https://github.com/cmacdonald/ColBERT.git to commit ec191156d9c18f9f057103d3f2c9576c3eba2595
  Installing build dependencies: started
  Installing build dependencies: fi

  Running command git clone --filter=blob:none --quiet https://github.com/terrierteam/pyterrier_colbert.git 'C:\Users\Konstantin-Asen\AppData\Local\Temp\pip-req-build-g5umfp1w'
  Running command git clone --filter=blob:none --quiet https://github.com/cmacdonald/ColBERT.git 'C:\Users\Konstantin-Asen\AppData\Local\Temp\pip-install-7jbxgzfq\colbert_3d6df633af30467681aa8ba52caabd0e'
  Running command git checkout -b v0.2 --track origin/v0.2
  branch 'v0.2' set up to track 'origin/v0.2'.
  Switched to a new branch 'v0.2'


The code below preferably should print True before running the experiments; if it's True, it will run faster

In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [2]:
import pyterrier as pt
import pandas as pd

In [3]:
msmarco = pt.get_dataset("msmarco_passage")
dataset = list(msmarco.get_corpus_iter())[:1000] # SMALL DATASET

print("First document:", dataset[0] if len(dataset) > 0 else "Dataset is empty!")

First document: {'docno': '0', 'text': 'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.\n'}


In [4]:
import os

marco_index_src = os.path.abspath("msmarco-index")
if not os.path.exists(marco_index_src):
    print("Index not found. Creating a new index...")
    # Create index with metadata
    indexer = pt.IterDictIndexer(marco_index_src, blocks=True, meta={"docno": 20, "text": 128})

    index_ref = indexer.index(dataset, fields=["docno", "text"])
    
    # TOGGLE FOR FULL DATASET (below)
    # index_ref = indexer.index(msmarco.get_corpus_iter(), fields=["docno", "text"])

    print("Index successfully created!")

index = pt.IndexFactory.of(marco_index_src)

Index not found. Creating a new index...


Java started (triggered by TerrierIndexer.__init__) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


Index successfully created!


In [5]:
bm25 = pt.terrier.Retriever(index, wmodel="BM25", metadata=["docno", "text"])
rm3 = pt.rewrite.RM3(index)

In [6]:
queries = msmarco.get_topics("dev.small")[:1000]

# TOGGLE FOR FULL DATASET (below)
# queries = msmarco.get_topics("dev.small")

print(f"Checking Queries:\n{queries.head()}")

qrels = msmarco.get_qrels("dev.small")[:1000]

# TOGGLE FOR FULL DATASET (below)
# qrels = msmarco.get_qrels("dev.small")

print(f"Checking Result Relevance:\n{qrels.head()}")

Checking Queries:
       qid                                          query
0  1048585                   what is paula deen s brother
1        2                       androgen receptor define
2   524332  treating tension headaches without medication
3  1048642                            what is paranoid sc
4   524447            treatment of varicose veins in legs
Checking Result Relevance:
      qid    docno  label
0  300674  7067032      1
1  125705  7067056      1
2   94798  7067181      1
3    9083  7067274      1
4  174249  7067348      1


In [7]:
bm25(queries)

,qid,docid,docno,text,rank,score,query
0,1048585,11,11,punishment designed to repair the damage done ...,0,8.930336,what is paula deen s brother
1,1048585,44,44,Medical Tours costa Rica office remains within...,1,7.914070,what is paula deen s brother
2,1048585,245,245,John Maynard Keynes was born at 7 Melville Roa...,2,6.843275,what is paula deen s brother
3,1048585,573,573,"After clicking on Email, it reports that there...",3,6.678970,what is paula deen s brother
4,1048585,623,623,Meet the dog star who plays heroic 'Max'. A bo...,4,5.385839,what is paula deen s brother
...,...,...,...,...,...,...,...
35841,1083158,924,924,Served with a splash of olive oil and a sprink...,2,7.372727,what does qmap stand for
35842,1083158,282,282,Please complete and return to the Student Gove...,3,6.730017,what does qmap stand for
35843,1083158,309,309,OHC Supply: The Total Builders Resource. OHC S...,4,6.190377,what does qmap stand for
35844,1083158,629,629,You can find out what district you vote in by ...,5,5.611833,what does qmap stand for


In [8]:
rm3(bm25(queries))

,qid,query_0,query
0,1004258,when he was born ben franklin,applypipeline:off ag^0.024823288 latrob^0.0496...
1,1005595,when dividing what is the quotient,applypipeline:off third^0.029678810 divid^0.34...
2,1006922,which chromosome controls sex characteristics,applypipeline:off sex^0.207142860 sorrento^0.0...
3,1007473,sediments originate from elements in seawater ...,applypipeline:off path^0.054962013 sediment^0....
4,1019649,what study for mets to brain,applypipeline:off serotonin^0.067361750 brain^...
...,...,...,...
882,934889,wheat germ oil skin benefits,applypipeline:off budder^0.048389226 shatter^0...
883,9454,a simple definition for the word bias,applypipeline:off mean^0.042057980 languag^0.0...
884,952520,when is the best time to visit spain,applypipeline:off degre^0.037751552 window^0.0...
885,961705,when was the little prince book lished,applypipeline:off tripadvisor^0.073430538 prin...


In [9]:
from pyterrier.measures import MAP, nDCG

pt.Experiment(
    [bm25, bm25 >> rm3 >> bm25],
    queries,
    qrels,
    names=["BM25", "BM25 >> RM3 >> BM25"],
    eval_metrics=[MAP, nDCG @ 10],
)

,name,AP,nDCG@10
0,BM25,0.0,0.0
1,BM25 >> RM3 >> BM25,0.0,0.0


In [ ]:
# For the ColBERT cell to work, replace the following file with the one from replacement-files/:
# .venv/Lib/site-packages/pyterrier_colbert/indexing.py
# Note: restarting VSCode is also necessary for the .venv to get updated properly

# More details: The replacement-file has the compute_throughput() usages commented out
# (the method was solely used in print-commands, so no core functionality is affected)
# (it was throwing ZeroDivisionError via the subtracted timestamps in the denominator)

In [11]:
from pyterrier_colbert.indexing import ColBERTIndexer
from pyterrier_colbert.ranking import ColBERTFactory
import wget
import zipfile

checkpoint_url = "http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip"
extract_dir = "colbert_checkpoint"
checkpoint_path = "colbert_checkpoint.zip"

if not os.path.exists(checkpoint_path):
    print("Downloading checkpoint...")
    wget.download(checkpoint_url, checkpoint_path)
if not os.path.exists(extract_dir):
    with zipfile.ZipFile(checkpoint_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

colbert_checkpoint_path = os.path.abspath("colbert_checkpoint/colbert.dnn")
index_root = os.path.abspath("msmarco-index")
index_name = os.path.abspath("colbert-index")

colbert_index = ColBERTIndexer(
    checkpoint=colbert_checkpoint_path,
    index_root=index_root,
    index_name=index_name,
    chunksize=64,
    gpu=True # if the torch.cuda returned False, comment this
)
colbert_index.index(dataset)

# TOGGLE FOR FULL DATASET (below)
# colbert_index.index(msmarco.get_corpus_iter())

[Mar 20, 21:08:08] [0] 		 #> Local args.bsize = 128
[Mar 20, 21:08:08] [0] 		 #> args.index_root = c:\Users\Konstantin-Asen\Desktop\IR-research-project\msmarco-index
[Mar 20, 21:08:08] [0] 		 #> self.possible_subset_sizes = [2982616]


Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Mar 20, 21:08:11] #> Loading model checkpoint.
[Mar 20, 21:08:11] #> Loading checkpoint c:\Users\Konstantin-Asen\Desktop\IR-research-project\colbert_checkpoint\colbert.dnn
[Mar 20, 21:08:11] #> checkpoint['epoch'] = 0
[Mar 20, 21:08:11] #> checkpoint['batch'] = 44500




[Mar 20, 21:08:12] #> Note: Output directory c:\Users\Konstantin-Asen\Desktop\IR-research-project\msmarco-index already exists




[Mar 20, 21:08:12] #> Creating directory c:\Users\Konstantin-Asen\Desktop\IR-research-project\colbert-index 


[Mar 20, 21:08:34] [0] 		 [NOTE] Done with local share.
[Mar 20, 21:08:34] [0] 		 #> Joining saver thread.
#> num_embeddings = 72119
[Mar 20, 21:08:34] #> Starting..
[Mar 20, 21:08:34] #> Processing slice #1 of 1 (range 0..1).
[Mar 20, 21:08:34] #> Will write to c:\Users\Konstantin-Asen\Desktop\IR-research-project\colbert-index\ivfpq.100.faiss.
[Mar 20, 21:08:34] #> Loading c:\Users\Konstantin-Asen\Desktop\IR-research-project\colbert-index\0.sample ...
#> Sample has shape (3605, 1

In [12]:
pytcolbert = ColBERTFactory(colbert_checkpoint_path, index_root, index_name)

Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Mar 20, 21:08:39] #> Loading model checkpoint.
[Mar 20, 21:08:39] #> Loading checkpoint c:\Users\Konstantin-Asen\Desktop\IR-research-project\colbert_checkpoint\colbert.dnn
[Mar 20, 21:08:39] #> checkpoint['epoch'] = 0
[Mar 20, 21:08:39] #> checkpoint['batch'] = 44500


c:\Users\Konstantin-Asen\Desktop\IR-research-project\.venv\Lib\site-packages\colbert\utils\amp.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [13]:
sparse_colbert = bm25 >> pytcolbert.text_scorer()

In [14]:
pt.Experiment(
    [bm25, sparse_colbert],
    queries,
    qrels,
    names=["BM25", "BM25 >> ColBERT"],
    eval_metrics=[MAP, nDCG @ 10]
)

c:\Users\Konstantin-Asen\Desktop\IR-research-project\.venv\Lib\site-packages\colbert\utils\amp.py:32: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else nullcontext()


,name,AP,nDCG@10
0,BM25,0.0,0.0
1,BM25 >> ColBERT,0.0,0.0
